# User Item Biases With Regularization
* Prediction for user $i$ and item $j$ is $\tilde r_{ij} = u_i + a_j$
* Loss function is $L = \sum_{\Omega}w_{ij}\text{loss}(r_{ij}, \tilde r_{ij}) + \lambda_u \sum_i (u_i - \bar u) ^2 + \lambda_a \sum_j (a_j - \bar a)^2 $
* $\bar u$ is the mean of $u_i$ and $\bar a$ is the mean of $a_j$ 
* $\Omega$ is the set of oberved pairs $(i, j)$
* $r_{ij}$ is the rating for user $i$ and item $j$
* $w_{ij}$ is the weight for the prediction $r_{ij}$ and is modeled as a power-law in the number of items seen by $i$ and users than have seen $j$: $w_{ij} = |j' : (i, j') \in \Omega| ^ {\lambda_{wu}} |i' : (i', j) \in \Omega| ^ {\lambda_{wa}}$
* $\text{loss}$ is mean squared error

In [1]:
const name = "ExplicitUserItemBiases"
const implicit = false;

In [2]:
import NBInclude: @nbinclude
import SparseArrays: sparse
import Statistics: mean
@nbinclude("Alpha.ipynb");

In [3]:
const training = get_split("training", implicit)
const validation = get_split("validation", implicit);

## Alternating Least Squares
* Given some hyperparameters $\lambda$, we can solve for $U$ and $A$ via Alternating Least Squares
* This is an iterative algorithm where we fix $A$, then solve for the $U$ that minimizes the loss function
* Then we fix $U$ and solve for the best $A$
* These two steps are repeated until the matrices $U$ and $A$ converge

## ALS for Explicit data
* If we fix $a$, then for each user $i$, $u_i$ is optimized when
* $u_i = \dfrac{\sum_{j \in \Omega_i}(r_{ij} - a_j) w_{ij} + \bar u \lambda_u}{ \sum_{j \in \Omega_i} w_{ij} + \lambda_u}$
* $\Omega$ is the set of (user, item) pairs that we have ratings for
* $\Omega_i$ is subset of $\Omega$ for which the user is the $i$-th user

In [4]:
function make_prediction(users, items, u, a)
    r = Array{eltype(u)}(undef, length(users))
    Threads.@threads for i = 1:length(r)
        @inbounds r[i] = u[users[i]] + a[items[i]]
    end
    r
end;

In [5]:
function get_residuals!(users, items, ratings, weights, a, ρ, Ω)
    @inbounds for row = 1:length(users)
        i = users[row]
        j = items[row]
        r = ratings[row]
        w = weights[row]
        ρ[i] += (r - a[j]) * w
        Ω[i] += w
    end
    ρ, Ω
end

function update_users!(users, items, ratings, weights, u, a, λ_u, ρ, Ω)
    Threads.@threads for t = 1:Threads.nthreads()
        range = thread_range(length(ratings))
        ρ[:, Threads.threadid()] .= 0
        Ω[:, Threads.threadid()] .= 0
        @views get_residuals!(
            users[range],
            items[range],
            ratings[range],
            weights[range],
            a,
            ρ[:, Threads.threadid()],
            Ω[:, Threads.threadid()],
        )
    end

    ρ = sum(ρ, dims = 2)
    Ω = sum(Ω, dims = 2)
    μ = mean(u)
    Threads.@threads for i = 1:length(u)
        @inbounds u[i] = (ρ[i] + μ * λ_u) / (Ω[i] + λ_u)
    end
end;

In [6]:
function train_model(training, stop_criteria, λ)
    @info "training model with parameters $λ"
    λ_u, λ_a, λ_wu, λ_wa = λ
    users, items, ratings = training.user, training.item, training.rating
    weights =
        expdecay(get_counts("training", implicit), log(λ_wu)) .*
        expdecay(get_counts("training", implicit; by_item = true), log(λ_wa))
    u = zeros(eltype(λ_u), num_users())
    a = zeros(eltype(λ_a), num_items())

    ρ_u = zeros(eltype(u), length(u), Threads.nthreads())
    Ω_u = zeros(eltype(u), length(u), Threads.nthreads())
    ρ_a = zeros(eltype(a), length(a), Threads.nthreads())
    Ω_a = zeros(eltype(a), length(a), Threads.nthreads())

    while !stop!(stop_criteria, [u, a])
        update_users!(users, items, ratings, weights, u, a, λ_u, ρ_u, Ω_u)
        update_users!(items, users, ratings, weights, a, u, λ_a, ρ_a, Ω_a)
    end
    u, a
end;

## Training

In [7]:
function validation_mse(λ)
    λ = exp.(λ) # ensure λ is nonnegative
    stop_criteria = convergence_stopper(1e-6, max_iters = 16)
    u, a = train_model(training, stop_criteria, λ)
    r = make_prediction(validation.user, validation.item, u, a)
    residualized_loss([], implicit, r)
end;

In [8]:
# Find the best regularization hyperparameters
res = Optim.optimize(
    validation_mse,
    fill(0.0f0, 4),
    Optim.BFGS(),
    autodiff = :forward,
    Optim.Options(show_trace = true, extended_trace = true),
);
λ = exp.(Optim.minimizer(res));

[ Info: 20220621 08:07:11 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,1.0,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,1.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,1.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0,0.0,0.0,0.0,1.0)]
Progress: 100%|███████████████████████████| Time: 0:00:00 (83.53 ns/it)it)


Iter     Function value   Gradient norm 
     0     1.813422e+00     5.013248e-02
 * Current step size: 1.0
 * time: 0.009202003479003906
 * g(x): Float32[-0.004593451, -3.0179515f-6, 0.018213123, 0.050132476]
 * ~inv(H): Float32[1.0 0.0 0.0 0.0; 0.0 1.0 0.0 0.0; 0.0 0.0 1.0 0.0; 0.0 0.0 0.0 1.0]
 * x: Float32[0.0, 0.0, 0.0, 0.0]


[ Info: 20220621 08:07:32 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.004604,1.004604,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.000003,0.0,1.000003,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9819517,0.0,0.0,0.9819517,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9511034,0.0,0.0,0.0,0.9511034)]
[ Info: 20220621 08:07:45 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.023233,1.023233,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000151,0.0,1.0000151,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9129578,0.0,0.0,0.9129578,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7782851,0.0,0.0,0.0,0.7782851)]
[ Info: 20220621 08:07

     1     1.811670e+00     1.526936e-02
 * Current step size: 1.0274389
 * time: 39.213566064834595
 * g(x): Float32[-0.0010373178, -3.792534f-6, 0.01526936, -0.0015094323]
 * ~inv(H): Float32[1.0038893 7.92986f-6 -0.03469486 -0.020281833; 7.92986f-6 1.0 -4.4105018f-5 -7.198241f-5; -0.034694858 -4.4105018f-5 1.2139859 0.29076782; -0.020281829 -7.198241f-5 0.29076782 0.97943676]
 * x: Float32[0.00471949, 3.1007608f-6, -0.01871287, -0.051508054]


[ Info: 20220621 08:08:11 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0062796,1.0062796,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000075,0.0,1.0000075,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.96382385,0.0,0.0,0.96382385,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9469675,0.0,0.0,0.0,0.9469675)]
[ Info: 20220621 08:08:24 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0124995,1.0124995,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(1.0000249,0.0,1.0000249,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.8963877,0.0,0.0,0.8963877,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9357373,0.0,0.0,0.0,0.9357373)]
[ Info: 202206

...

    15     1.808034e+00     1.518278e-06
 * Current step size: 1.187046
 * time: 627.2488920688629
 * g(x): Float32[1.5653112f-7, 4.317415f-9, -1.5182777f-6, -1.4051899f-6]
 * ~inv(H): Float32[746.0159 274.41168 60.586273 48.460144; 274.4121 69479.305 47.48637 15.69807; 60.586266 47.486366 19.163885 0.8995602; 48.460144 15.698062 0.8995618 4.2420607]
 * x: Float32[-0.3444668, -0.9052477, -0.4290608, -0.018738288]


[ Info: 20220621 08:17:59 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.70862794,0.70862794,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.4043411,0.0,0.4043411,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.65113384,0.0,0.0,0.65113384,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98143584,0.0,0.0,0.0,0.98143584)]
[ Info: 20220621 08:18:12 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.70862323,0.70862323,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.404357,0.0,0.404357,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.6511317,0.0,0.0,0.6511317,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.9814359,0.0,0.0,0.0,0.9814359)]


    16     1.808034e+00     7.718816e-07
 * Current step size: 0.84230554
 * time: 652.9967639446259
 * g(x): Float32[4.863387f-8, 1.4080562f-10, -1.9309024f-7, -7.7188156f-7]
 * ~inv(H): Float32[775.79315 229.5718 66.60399 50.342762; 229.57222 68970.36 76.24287 3.563901; 66.60398 76.242874 17.899004 1.8900824; 50.342762 3.5638928 1.890084 4.211076]
 * x: Float32[-0.3444313, -0.9054572, -0.42904338, -0.018738564]


[ Info: 20220621 08:18:25 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.7086331,0.7086331,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.4043556,0.0,0.4043556,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.65113276,0.0,0.0,0.65113276,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.981437,0.0,0.0,0.0,0.981437)]
[ Info: 20220621 08:18:38 training model with parameters ForwardDiff.Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}, Float32, 4}[Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.70862776,0.70862776,0.0,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.40435633,0.0,0.40435633,0.0,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.65113217,0.0,0.0,0.65113217,0.0), Dual{ForwardDiff.Tag{typeof(validation_mse), Float32}}(0.98143643,0.0,0.0,0.0,0.98143643)]


    17     1.808034e+00     5.600174e-07
 * Current step size: 0.4571615
 * time: 678.628494977951
 * g(x): Float32[2.7551085f-8, 2.9163533f-11, -8.036908f-10, -5.600174f-7]
 * ~inv(H): Float32[1311.246 -136.04721 49.66346 115.453926; -136.04681 69116.836 52.33392 -31.959457; 49.66346 52.33393 6.2359567 2.9028714; 115.45393 -31.959463 2.9028735 11.354621]
 * x: Float32[-0.34442493, -0.90545875, -0.4290426, -0.018738031]


In [9]:
@info "The optimal λ is $λ, found in " * repr(Optim.f_calls(res)) * " function calls"

[ Info: 20220621 08:18:50 The optimal λ is Float32[0.70862776, 0.40435633, 0.65113217, 0.98143643], found in 54 function calls


In [10]:
stop_criteria = convergence_stopper(1e-6, max_iters = 16)
u, a = train_model(training, stop_criteria, λ);

[ Info: 20220621 08:18:50 training model with parameters Float32[0.70862776, 0.40435633, 0.65113217, 0.98143643]


In [11]:
validation_mse(Optim.minimizer(res))

[ Info: 20220621 08:18:55 training model with parameters Float32[0.70862776, 0.40435633, 0.65113217, 0.98143643]


1.8080269f0

## Inference

In [12]:
model(users, items) = make_prediction(users, items, u, a)
write_alpha(model, [], implicit, name);

[ Info: 20220621 08:19:02 validation loss: 1.8080269, β: Float32[1.0015494]
[ Info: 20220621 08:19:04 training loss: 1.6717248, β: Float32[1.0015494]


In [13]:
write_params(Dict("u" => u, "a" => a, "λ" => λ), name);